In [1]:
import pandas as pd

In [2]:
degree = pd.read_csv('F:/bulk_export_122020/degrees.csv')
organization = pd.read_csv('F:/bulk_export_122020/organizations.csv')
funding_round = pd.read_csv('F:/bulk_export_122020/funding_rounds.csv')
people = pd.read_csv('F:/bulk_export_122020/people.csv')

In [15]:
org = pd.DataFrame(organization,columns=['uuid','status','num_funding_rounds','total_funding_usd','founded_on','closed_on','employee_count'])
fr = pd.DataFrame(funding_round,columns=['investment_type','raised_amount','investor_count','org_uuid'])
dg = pd.DataFrame(degree,columns=['person_uuid','person_name','degree_type','subject','is_completed'])
people = pd.DataFrame(people,columns=['uuid','featured_job_organization_uuid','gender','rank'])

In [16]:
org['years'] = ''
org['highest_degree'] = ''
org['number_bs'] = ''
org['number_msc'] = ''
org['number_phd'] = ''
org['number_mbc'] = ''
org['number_investor'] = ''
org['number_founder'] = ''

In [17]:
org

,uuid,status,num_funding_rounds,total_funding_usd,founded_on,closed_on,employee_count,years,highest_degree,number_bs,number_msc,number_phd,number_mbc,number_investor,number_founder
0,e1393508-30ea-8a36-3f96-dd3226033abd,acquired,3.0,3.975000e+07,2005-06-01,NaN,51-100,,,,,,,,
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,operating,NaN,NaN,1996-09-15,NaN,1001-5000,,,,,,,,
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,acquired,6.0,4.900000e+07,2004-10-11,NaN,51-100,,,,,,,,
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,operating,NaN,NaN,2004-01-01,NaN,101-250,,,,,,,,
4,df662812-7f97-0b43-9d3e-12f64f504fbb,ipo,17.0,1.612282e+10,2004-02-04,NaN,10000+,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216440,09c8e895-e62e-46f0-93f9-45e088ede007,operating,NaN,NaN,2020-09-08,NaN,unknown,,,,,,,,
1216441,9b7c50d5-8d96-4af4-9b0e-af72705184e9,operating,NaN,NaN,2015-01-01,NaN,1-10,,,,,,,,
1216442,98589112-e90c-434b-a4e7-ffaa12eee602,operating,1.0,2.350000e+06,NaN,NaN,unknown,,,,,,,,
1216443,90418618-ce0c-49b2-8c0b-1f741a3d313e,operating,NaN,NaN,1952-01-01,NaN,51-100,,,,,,,,


In [18]:
print(org.loc[:,'status'].unique())
org = org.loc[org.loc[:,'founded_on'] > '2012-01-01']
org = org.loc[org['num_funding_rounds'] < 4]
# org.loc[org['uuid'] == '231e0f6b-8bdf-28b0-a82c-b42bdec2c67d']['roles'].values
# org['is_company'] = org['roles'].map(lambda x:x.split(',')[0])
# org['is_investor'] = org['roles'].map(lambda x:x.split(',')[1])
# print(org['roles'].unique())
# org['roles'] = org['roles'].map({'company':0, 'investor':1, 'investor,company':2})

org.loc[:, 'status'] = org.loc[:, 'status'].map({'acquired':1, 'operating':1, 'ipo':1,'closed':0})
org.loc[:, 'num_funding_rounds'] = org.loc[:, 'num_funding_rounds'].fillna(0)

# print(org['employee_count'].unique())
# org['employee_count'] = org['employee_count'].map({'101-250':3, '51-100':2, '11-50':1, '1-10':0, '5001-10000':7, '10000+':8,
#        'unknown':9, '251-500':4, '1001-5000':6, '501-1000':5})
org.loc[:, 'employee_count'] = org.loc[:, 'employee_count'].map({'101-250':1, '51-100':0, '11-50':0, '1-10':0, '5001-10000':2, '10000+':2,
       'unknown':3, '251-500':1, '1001-5000':2, '501-1000':1})

########calculate the years###################
# int('2021-01-01'.split('-')[0])
org.loc[:, 'closed_on'] = org.loc[:, 'closed_on'].fillna('2021-01-01')
# org['founded_on'] = org.founded_on.apply(lambda x: int(x.split('-')[0]))
org.loc[:,'years'] = org.loc[:, 'closed_on'].apply(lambda x: int(x.split('-')[0])) - org.loc[:, 'founded_on'].apply(lambda x: int(x.split('-')[0]))


org.loc[:, 'total_funding_usd'] = org.loc[:, 'total_funding_usd'].fillna(0)
print(org['total_funding_usd'].max())


['acquired' 'operating' 'ipo' 'closed']


In [57]:

org2 = org.loc[:, ['status','total_funding_usd','employee_count']] 
org2 = org2.drop(org2[org2['employee_count']==0].index)
org2 = org2.drop(org2[org2['total_funding_usd']==0].index)
one_hot_em = pd.get_dummies(org2['employee_count'])
one_hot_em.columns = ['employee_count_1hot_100','employee_count_1hot_1000','employee_count_1hot_10000']
org2 = org2.join(one_hot_em)
# org2 = org2.rename(columns={'1': 'employee_count_1hot_100'})
# org2 = org2.rename(columns={'2': 'employee_count_1hot_1000'})
# org2 = org2.rename(columns={'3': 'employee_count_1hot_10000'})
# org2
def my_cut(x):
    if 0 <= x < 1000000 :
        return 1
    if 1000000 <= x < 10000000:
        return 2
    if 10000000 <= x < 100000000:
        return 3
    else:
        return 4
org2.loc[:, 'total_funding_usd'] = org2.loc[:, 'total_funding_usd'].map(my_cut)
one_hot_fu = pd.get_dummies(org2['total_funding_usd'])
one_hot_fu.columns = ['total_funding_1hot_100','total_funding_1hot_1000','total_funding_1hot_10000','total_funding_1hot_l_10000']
org2 = org2.join(one_hot_fu)
org2 = org2.drop(columns=['total_funding_usd','employee_count'])
org2.to_csv('data_v1.csv',index=False)

In [44]:
print(org2.groupby(['employee_count'],as_index=False)['employee_count'].agg({'cnt':'count'}))
print(org2.groupby(['status'],as_index=False)['status'].agg({'cnt':'count'}))

   employee_count   cnt
0               1  3257
1               2   305
2               3  1947
   status   cnt
0       0   552
1       1  4957


In [44]:
dg['degree_type'].astype('string')
print(dg.groupby(['degree_type'],as_index=False)['degree_type'].agg({'cnt':'count'}))
print(len(dg['degree_type'].unique()))
dg2 = dg.groupby(by='person_uuid').apply(lambda x:[','.join(map(str,x['degree_type']))]).to_frame()
dg2

15655


,0
person_uuid,
000020dc-18ce-7f7b-e8c4-8f5d716ad09d,"[B.A.,M.S.W.,Ph.D.]"
00002852-4f2a-473a-ae63-810fa8d3f31f,[BS]
00006aa5-68cc-7430-eb3d-9bd8305dcb4d,"[MBA,BS]"
0000a1a4-5804-cf06-d23e-cecc8017b220,"[unknown,unknown]"
0000bf3d-7645-e2ef-ec7b-0408a9b2f879,"[B.Com,B.Com]"
...,...
fffedd65-2f76-4175-80db-e516c8357794,[nan]
fffee386-822f-4f78-a61d-6ee443ee0c15,[BBA]
fffefc19-5a64-6535-17c2-3155be76a204,[BS]


In [41]:

# dg2['000020dc-18ce-7f7b-e8c4-8f5d716ad09d']
print(dg2['0000bf3d-7645-e2ef-ec7b-0408a9b2f879'])
# dg2
# people['degree'] = ''
# personlist = dg2.index.tolist()
# for r1 in range(people.shape[0]):
#     for i in range(len(personlist)):
#          if(people.loc[r1]['uuid'] == personlist[i]):
#             print(i)
#             people.loc[r1]['degree'] = dg2[personlist[i]]
# people

KeyError: '0000bf3d-7645-e2ef-ec7b-0408a9b2f879'

In [119]:
people2 = people.groupby(by='featured_job_organization_uuid').apply(lambda x:[','.join(map(str,x['gender']))]).to_frame()
# people2.columns = ['degrees']
people2['number_founder'] = ''
len(people2.iloc[0,0][0].split(','))
# len(people2)
for r1 in range(len(people2)):
    people2.iloc[r1,1] = len(people2.iloc[r1,0][0].split(','))
# people2.iloc[0]
# people2['number_founder'] = people2.apply(lambda x: len(x.iloc[:,0][0].split(',')))

In [120]:
people2

,0,number_founder
featured_job_organization_uuid,,
00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,[male],1
000014da-0c46-b9cb-0941-3a93c027b119,"[male,male,female,male,male]",5
00002470-bff7-6226-5800-0ca1b3787b6f,"[male,male,male]",3
00004c41-aeeb-b5a0-8812-64cf2d017cfb,"[male,male,male,female]",4
00007c5c-9260-0dfb-c160-89a416f1a7cc,"[male,male]",2
...,...,...
fffec51c-9609-fa00-cc8a-4fa7905b6e62,[female],1
ffff2c3b-ca66-67dc-c643-77cf3a5f89e2,"[female,male,female]",3
ffff5bda-7616-2595-10df-fc0e8e7b08de,[female],1


In [86]:
# aggregate the investment_type
print(len(fr['investment_type'].unique()))
data1 = fr.groupby(by='org_uuid').apply(lambda x:[','.join(x['investment_type'])])
data1 

28


org_uuid
00000aa4-ba42-9b68-a9c3-040c9f3bf9b9                                     [series_unknown]
000014da-0c46-b9cb-0941-3a93c027b119    [pre_seed,grant,grant,convertible_note,pre_see...
00002470-bff7-6226-5800-0ca1b3787b6f                 [seed,seed,seed,series_unknown,seed]
000095de-8e2b-82f1-32a7-c222ba3d5682                                     [series_unknown]
0000d497-c93a-eea3-eeb0-a943dfb4f71e                                [seed,series_unknown]
                                                              ...                        
fffdb9e1-0ae9-4630-be83-b76e9e6e0ca0                                           [pre_seed]
fffe2508-e9e3-8602-9664-8c6ef2c803de                                               [seed]
fffe3b0b-8545-4414-9c0e-a3f961c4e420                                     [series_unknown]
ffff25bc-9902-33e8-062e-6d84841d8cf9                                     [series_unknown]
ffffabce-6d4a-b3d1-13c0-4e90cedf5270    [angel,grant,product_crowdfunding,seed,seed,un...
L

In [87]:
data1['df662812-7f97-0b43-9d3e-12f64f504fbb']


['angel,series_a,series_b,series_c,series_c,series_c,debt_financing,series_c,series_d,secondary_market,private_equity,secondary_market,secondary_market,private_equity,post_ipo_secondary,secondary_market,post_ipo_equity']

In [93]:
# org['investment_types'] = df.apply(lambda x: x.amount if x.name != "" else 0, axis=1)
org.loc[org['num_funding_rounds'] > 3]
# print(org['status'].unique())

,uuid,roles,city,status,category_list,category_groups_list,num_funding_rounds,total_funding_usd,founded_on,employee_count
282,ed54b2d5-f2f1-d4e9-9bbe-40961ff08d44,company,Foster City,acquired,"Apps,Consulting,Mobile,Software,UX Design,Web ...","Apps,Design,Mobile,Professional Services,Software",5.0,5.820000e+07,2020-02-01,1
1179,74a20af3-f4dd-6188-de60-c4ee6cd0ca4a,"investor,company",Hangzhou,operating,"Banking,E-Commerce,Financial Services,FinTech,...","Commerce and Shopping,Financial Services,Inter...",4.0,2.200000e+10,2014-10-01,4
2562,1966bb69-4bc9-1077-c181-fe3d67509160,"investor,company",Hoofddorp,acquired,"E-Commerce,Internet,Shopping","Commerce and Shopping,Internet Services",5.0,2.850000e+07,2015-01-01,1
3935,1ebaa59b-7035-75d2-316d-e7b35b8a82ed,company,New York,ipo,"Artificial Intelligence,Financial Services,Ins...","Artificial Intelligence,Data and Analytics,Fin...",7.0,4.800000e+08,2015-01-01,7
5760,b76dac89-a8f8-abe1-c129-f64bfe7fa3c1,company,Arlington,operating,"Data Integration,Enterprise Software,Semantic ...","Data and Analytics,Information Technology,Inte...",5.0,2.330000e+07,2015-12-01,1
...,...,...,...,...,...,...,...,...,...,...
1193713,1212a3d6-c5c6-4fc3-94ef-bee2c7f86408,company,Roma,operating,"Apps,FinTech,Mobile Apps,Mobile Payments,Payments","Apps,Financial Services,Mobile,Payments,Software",5.0,1.032811e+06,2018-01-01,3
1195131,2c7fdbbe-57d5-48a7-8ce8-218dd1c2ff01,investor,Los Angeles,operating,"Dental,Medical,Software","Health Care,Software",4.0,5.265000e+05,2019-01-01,3
1195137,e5416a37-0960-4da3-b059-f4e3adf8086d,company,NaN,operating,"Health Diagnostics,Semiconductor","Hardware,Health Care,Science and Engineering",4.0,3.749840e+05,2018-01-01,3
1201762,90753ea4-8620-497b-b91b-21299427d217,company,New York,acquired,"Apps,Internet,Online Portals,Payments,Professi...","Apps,Financial Services,Internet Services,Othe...",4.0,0.000000e+00,2014-01-01,0


In [90]:


# find the series _C organization
fr1 = fr.loc[fr['investment_type'] == 'series_b']
list1 = fr1['org_uuid'].values.tolist()
print(len(list1))
org2 = org.loc[org['uuid'].isin(list1)]

org2

21084


,uuid,roles,city,status,category_list,category_groups_list,num_funding_rounds,total_funding_usd,founded_on,employee_count
282,ed54b2d5-f2f1-d4e9-9bbe-40961ff08d44,company,Foster City,acquired,"Apps,Consulting,Mobile,Software,UX Design,Web ...","Apps,Design,Mobile,Professional Services,Software",5.0,5.820000e+07,2020-02-01,1
1179,74a20af3-f4dd-6188-de60-c4ee6cd0ca4a,"investor,company",Hangzhou,operating,"Banking,E-Commerce,Financial Services,FinTech,...","Commerce and Shopping,Financial Services,Inter...",4.0,2.200000e+10,2014-10-01,4
2562,1966bb69-4bc9-1077-c181-fe3d67509160,"investor,company",Hoofddorp,acquired,"E-Commerce,Internet,Shopping","Commerce and Shopping,Internet Services",5.0,2.850000e+07,2015-01-01,1
3935,1ebaa59b-7035-75d2-316d-e7b35b8a82ed,company,New York,ipo,"Artificial Intelligence,Financial Services,Ins...","Artificial Intelligence,Data and Analytics,Fin...",7.0,4.800000e+08,2015-01-01,7
5760,b76dac89-a8f8-abe1-c129-f64bfe7fa3c1,company,Arlington,operating,"Data Integration,Enterprise Software,Semantic ...","Data and Analytics,Information Technology,Inte...",5.0,2.330000e+07,2015-12-01,1
...,...,...,...,...,...,...,...,...,...,...
1193008,f02593fd-ee80-4831-aac7-975ca689c8f4,company,San Francisco,operating,"Health Care,Medical",Health Care,1.0,0.000000e+00,2018-01-01,6
1194361,c38873e3-8b9b-4ade-b939-09d77eaf5175,company,Seoul,operating,"Artificial Intelligence,Information Technology...","Artificial Intelligence,Data and Analytics,Inf...",2.0,5.407308e+06,2015-06-15,0
1199195,28bb185e-93a7-4d92-bac4-cfb77f34e701,company,La Jolla,operating,"Biopharma,Biotechnology,Life Science,Pharmaceu...","Biotechnology,Health Care,Science and Engineering",2.0,8.000000e+08,2020-01-01,6
1202177,5bec8ba0-c23c-48ec-87c1-54557e5b82ba,company,Yangju,operating,"Health Care,Medical Device",Health Care,1.0,1.761911e+06,2016-01-11,2
